In [1]:
from pyspark import SparkContext, SparkConf


# create a SparkContext object
conf = SparkConf().setAppName("GraphRDD")
sc = SparkContext(conf=conf)

# create a graph

edges_rdd = sc.parallelize([(0,1),(1,2),(1,3),(3,4),(5,6),(6,7),(7,8)])

# print the RDD
print(edges_rdd.collect())

[(0, 1), (1, 2), (1, 3), (3, 4), (5, 6), (6, 7), (7, 8)]


In [2]:
# Function for the reducer

def reducer(edges):
    key = edges[0]
    values = edges[1]
    minim = key
    valueList = []
    to_emit = []
    counter = 0
    for value in values:
        if value < minim:
            minim = value
        valueList.append(value)
    if minim < key:
        to_emit.append((key,minim))
        for value in valueList:
            if minim != value:
                counter += 1
                to_emit.append((value,minim))
    return to_emit, counter

In [3]:
# initialising count_new_pairs with a value > 0 to run the first iterartion

import time

start_time = time.time()

count_new_pairs = 1

iterations = 0

# Loop for the iterations

while count_new_pairs > 0:
    
    # CCF iterate
    
    # Map
    result_map = edges_rdd.flatMap(lambda x: ((x[0],x[1]),(x[1],x[0])))
    
    # Shuffle and Sort
    result_shufflesort = result_map.groupByKey().map(lambda x : (x[0], list(x[1])))
    
    # Reduce
    result_reducer = result_shufflesort.map(lambda x: reducer(x))
    result_reducer2 = result_reducer.flatMap(lambda x: x[0])
    
    # Updating the counting
    count_new_pairs = result_reducer.map(lambda x: x[1]).reduce(lambda x, y : x + y)
    
    # CCF dedup    
    edges_rdd = result_reducer2.distinct()
    
    iterations+=1

sorted_rdd = edges_rdd.sortBy(lambda x: x[1])
end_time = time.time()

elapsed_time = end_time - start_time

print(sorted_rdd.collect()) 
print("Iterations:", iterations)
print("Elapsed time: ", elapsed_time)

[(2, 0), (3, 0), (4, 0), (1, 0), (8, 5), (7, 5), (6, 5)]
Iterations: 4
Elapsed time:  2.3227078914642334
